In [1]:
# Import and install dependencies.
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
# We will setup Media Pipe Holistic

#This is mediapipe holistic model which makes the detections.
mp_holistic = mp.solutions.holistic

#This is mediapipe holistic drawing utilities which will draw the detection.
mp_drawing = mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #This is color coversion from BGR to RGB.
    image.flags.writeable = False  #image is no longer writeable.
    results = model.process(image) #Make prediction.
    image.flags.writeable = True  #Image is now writeable.
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #This is color conversion from RGB to BGR.
    return image, results

#### first we will access webcam using cv2 </br>
#### This is the method for accessing the web cam we will use this method to collect dataset for our model to train</br>
cap = cv2.VideoCapture(0)</br>
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        
        #capturig frames from the webcam
        ret, frame = cap.read()

        #between read and rendering we want to make our prediction.
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        #Draw ladmarks in real time
        enhanced_landmarks(image, results)

        #Showing frames on the screen.
        cv2.imshow('OpenCV Feed', image)

        #Breaking gracefully from the loop.
        #This line specifies that when we want to break out of loop we need to press 'q' on the keyboard.
        if cv2.waitKey(10) & 0XFFF == ord('q'):
            break;   
    cap.release()
    #This line of code closes alll the windows.
    cv2.destroyAllWindows()

#### Then on above that layer we will apply mediapipe <br>

In [5]:
def draw_landmarks(image, results):
    #Draw Face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    #Draw pose Connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    #Draw left Hand Connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    #Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245, 117, 76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [6]:
#This method is for extracting keypoints from our media pipe detection for further recognition.

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [13]:
#path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data')

In [14]:
#Actions that we try to detect
#First we will train our model for digits
actions = np.array(['Day', 'Morning', 'Afternoon', 'Evening', 'Night'])
#Thirty videos worth of data
no_sequences = 30

#videos are going to be 30 frames in length
sequence_length = 30

In [21]:
actions = np.array(['Hello','Woman','Thankyou', 'Thankyou Very Much','Bye','My name is', 'Correct', 'Wrong', 'Good']) #Done

In [22]:
#This traverse through every actions or labels and inside that action there will be thirty videos, each of thirty frames and 
#Creates the folder or directory for the same
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [23]:
#This method is for collecting keyframes or dataset for the acitons or lables specified in the actions variable above

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #Loop through actions
    for action in actions:
        #Loop through each videos
        for sequence in range(no_sequences):
            #Loop through video length
            for frame_num in range(sequence_length):
                #capturig frames from the webcam
                ret, frame = cap.read()

                #between read and rendering we want to make our prediction.
                image, results = mediapipe_detection(frame, holistic)
                print(results)

                #Draw ladmarks in real time
                draw_landmarks(image, results)

                #Apply wait Logic.
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} video Number {}'.format(action, sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                #Now we will extract keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num)) 
                np.save(npy_path, keypoints)


                #Showing frames on the screen.
                cv2.imshow('OpenCV Feed', image)

                #Breaking gracefully from the loop.
                #This line specifies that when we want to break out of loop we need to press 'q' on the keyboard.
                if cv2.waitKey(10) & 0XFFF == ord('q'):
                    break;  
                    
    cap.release()
    #This line of code closes alll the windows.
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [24]:
actions = np.array(['Day', 'Morning', 'Afternoon', 'Evening', 'Night', 'Hello','Woman','Thankyou', 'Thankyou Very Much','Bye','My name is', 'Correct', 'Wrong', 'Good'])

## preprocessing of data

In [25]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [26]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'Day': 0,
 'Morning': 1,
 'Afternoon': 2,
 'Evening': 3,
 'Night': 4,
 'Hello': 5,
 'Woman': 6,
 'Thankyou': 7,
 'Thankyou Very Much': 8,
 'Bye': 9,
 'My name is': 10,
 'Correct': 11,
 'Wrong': 12,
 'Good': 13}

In [27]:
#Now we will bring our data together and start structuring
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [28]:
np.array(sequences).shape

(420, 30, 1662)

In [29]:
np.array(labels).shape

(420,)

In [30]:
x = np.array(sequences)
x.shape

(420, 30, 1662)

In [31]:
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [32]:
#Now we will split our data into training and testing parts.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

In [33]:
x_train.shape

(399, 30, 1662)

In [34]:
x_test.shape

(21, 30, 1662)

In [35]:
y_train.shape

(399, 14)

In [36]:
y_test.shape

(21, 14)

## Creating The LSTM Neural Network Model

In [37]:
# Now we will Build the LSTM model

# For this purpose we are going to use tensorflow and keras
#first of all we have to import some dependencies that is 
#Sequencial model
#LSTM model
#Dense layer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard #This gonna allow some logins in tesorflow

In [38]:
#Now we will create a log directory and setup our tensorflow callbacks
#log directory allows us to monitor our neural network training
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [39]:
#Now we will build our neural network architecture
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [40]:
#Now we will compile our model and fit it
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [41]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
13/13 [==============================] - 4s 103ms/step - loss: 2.7443 - categorical_accuracy: 0.0788
Epoch 2/2000
13/13 [==============================] - 1s 74ms/step - loss: 2.5087 - categorical_accuracy: 0.1082
Epoch 3/2000
13/13 [==============================] - 1s 79ms/step - loss: 2.2303 - categorical_accuracy: 0.2830
Epoch 4/2000
13/13 [==============================] - 1s 78ms/step - loss: 2.6324 - categorical_accuracy: 0.2407
Epoch 5/2000
13/13 [==============================] - 1s 81ms/step - loss: 2.5854 - categorical_accuracy: 0.0820
Epoch 6/2000
13/13 [==============================] - 1s 79ms/step - loss: 2.3640 - categorical_accuracy: 0.2184
Epoch 7/2000
13/13 [==============================] - 1s 76ms/step - loss: 2.0882 - categorical_accuracy: 0.2231
Epoch 8/2000
13/13 [==============================] - 1s 76ms/step - loss: 2.1411 - categorical_accuracy: 0.2519
Epoch 9/2000
13/13 [==============================] - 1s 79ms/step - loss: 1.8558 - categorical

KeyboardInterrupt: 

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 14)                462       
Total params: 597,038
Trainable params: 597,038
Non-trainable params: 0
__________________________________________________

In [43]:
#del model

In [44]:
#Now Testing our model
res = model.predict(x_test)

In [46]:
actions[np.argmax(res[1])]

'Thankyou Very Much'

In [47]:
actions[np.argmax(y_test[1])]

'Thankyou Very Much'

In [48]:
model.save('action.h5')

In [49]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [50]:
yhat = model.predict(x_test)

In [51]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [52]:
yhat

[9, 8, 12, 8, 0, 2, 3, 11, 2, 13, 5, 7, 0, 6, 6, 9, 3, 2, 9, 9, 5]

In [53]:
ytrue

[9, 8, 12, 8, 0, 2, 4, 11, 2, 13, 5, 7, 0, 6, 6, 9, 3, 2, 9, 9, 5]

In [54]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[19,  0],
        [ 0,  2]],

       [[18,  0],
        [ 0,  3]],

       [[19,  1],
        [ 0,  1]],

       [[20,  0],
        [ 1,  0]],

       [[19,  0],
        [ 0,  2]],

       [[19,  0],
        [ 0,  2]],

       [[20,  0],
        [ 0,  1]],

       [[19,  0],
        [ 0,  2]],

       [[17,  0],
        [ 0,  4]],

       [[20,  0],
        [ 0,  1]],

       [[20,  0],
        [ 0,  1]],

       [[20,  0],
        [ 0,  1]]], dtype=int64)

In [55]:
accuracy_score(ytrue, yhat)

0.9523809523809523

## Test In Real Time

In [121]:
sequence = []
sentence = []
predictions = []
threshold = 0.4

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():

        #capturig frames from the webcam
        ret, frame = cap.read()

        #between read and rendering we want to make our prediction.
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        #Draw ladmarks in real time
        draw_landmarks(image, results)

        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30 :
            res = model.predict(np.expand_dims(sequence, axis = 0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
            #identify the unique sequence
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold :
                    if len(sentence) > 0 :
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5 :
                sentence = sentence[-5:]

        cv2.rectangle(image, (0,0), (640,40), (245,117,16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX , 1, (255,255,255), 2, cv2.LINE_AA)


        #Showing frames on the screen.
        cv2.imshow('OpenCV Feed', image)

        #Breaking gracefully from the loop.
        #This line specifies that when we want to break out of loop we need to press 'q' on the keyboard.
        if cv2.waitKey(10) & 0XFFF == ord('q'):
            break   
    cap.release()
    #This line of code closes alll the windows.
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [104]:
cap.release()
    #This line of code closes alll the windows.
cv2.destroyAllWindows()

In [109]:
np.unique(predictions[-10:])[0].size

1

In [119]:
print(sentence)

[]
